In [2]:
# 1.colab 환경 설정
!pip install transformers
!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
Cloning into 'korean-malicious-comments-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 30 (delta 8), reused 2 (delta 2), pack-reused 21
Receiving objects: 100% (30/30), 525.19 KiB | 21.88 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [3]:
!pip install transformers[torch]

!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [5]:
# GPU 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cuda:0


In [6]:
# 2 데이터셋 제작

In [7]:
# 2 - 1 원본 데이터 불러오기
df = pd.read_csv("/content/korean-malicious-comments-dataset/Dataset.csv", sep="\t")
df.head()

# 댓글과 레이블은 탭(\t)으로 구분되어 있으므로 set 파라미터로 "\t"를 사용해 데이터를 불러온다.

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [8]:
# 2 - 2 데이터 전처리
df.info()

# 텍스트를 나타내는 content 10,000개의 데이터가 정확하게 로드
# 악성댓글 여부를 나타내는 lable은 25개의 null데이터가 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [9]:
null_idx = df[df.lable.isnull()].index
df.loc[null_idx, "content"]


1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [10]:
# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[-2])

# 데이터를 불러올때 "\t"를 sep파리미터로 설정했지만 "\t"앞에 특수문자가 있는 경우 데이터가 정확학 분리되지 않는 경우가 발생
# 데이터를 정확하게 분리하기 위해 추가적인 전처리 작업 실행

In [11]:
# 학습을 위해 lable의 데이터 타입을 float가 아닌 int로 변환
df = df.astype({"lable":"int"})
df.info()

# 결측치가 없고 lable은 정수형인 데이터로 변환

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [12]:
# 2 - 3 Train set / Test set으로 나누기
# 모델을 학습할 Train data와 모델을 평가할 Test data 나눈다

train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [13]:
# 데이터셋에 존재하는 중복된 데이터를 제거

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["content"], inplace=True)
test_data.drop_duplicates(subset=["content"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 8000
중복 제거 전 테스트 데이터셋 : 2000
중복 제거 후 학습 데이터셋 : 7972
중복 제거 후 테스트 데이터셋 : 1997


In [14]:
# 2 - 4 토크나이징
# 텍스트 형태의 데이터를 모델이 학습할 수 있는 토큰 형태로 분리하고 토큰 id값으로 변환

MODEL_NAME = "beomi/KCELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# transformers 라이브러리의 AutoTokenizer 모듈을 활용해 손쉽게 진행

In [15]:
# 학습 데이터셋
# 토크나이저를 활용하여 데이터를 토크나이징
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",          # pytorch이 tensor 형태로 return
    max_length=128,               # 최대 토큰길이 설정
    padding=True,                 # 제로패딩 설정
    truncation=True,              # max_length 초과 토큰 truncate
    add_special_tokens=True,      # special token 추가
)

# tokenizer에 데이터를 입력하면 자동으로 토크나이징하는 과정을 수행 다양한 파라미터를 설정가능


In [16]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['ĠêµŃë°©ë¶Ģ', '~~', 'ìłĦ', 'íĻĶë¡ľ', 'Ġíľ´ê°Ģ', 'ìĹ°', 'ìŀ¥ìĿĦ', 'Ġíķľ', 'Ġë³ĳ', 'ìĤ¬ëĵ¤', 'ĠëªĩìĿ´ëĤĺ', 'ĠëĲĺëĬĶì§Ģ', 'Ġê³µê°ľíķ´ëĿ¼', '~', 'Ġìĸ´ëĬĲ', 'ĠíĽĮë¥Ńíķľ', 'Ġì§ĳìķĪ', 'ìĿĺ', 'ĠìŀĲìłľ', 'ë¶Ħëĵ¤', 'ìĿ¸ì§ĢëıĦ', 'Ġê°ĻìĿ´', 'Ġê³µê°ľíķ´ëĿ¼', '~~', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [17]:
# 테스트 데이터셋
tokenized_test_sentences = tokenizer(
    list(test_data["content"]),
    return_tensors="pt",          # pytorch이 tensor 형태로 return
    max_length=128,               # 최대 토큰길이 설정
    padding=True,                 # 제로패딩 설정
    truncation=True,              # max_length 초과 토큰 truncate
    add_special_tokens=True,      # special token 추가
)

In [18]:
# 2 - 5 데이터셋 생성
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [19]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["lable"].values
test_label = test_data["lable"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [20]:
# 3 모델 학습

In [21]:
# 3 - 1 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# 사전 학습된 kcELECTRA 모델을 불러온다

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KCELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [22]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './',            # 학습결과 저장경로
    num_train_epochs=10,          # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=100,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [23]:
# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

  # 정확도, 정밀도, 재현율, f1스코어   모델의 성능 평가

In [24]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [25]:
# 3 - 3 학습
trainer.train()

<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
100,0.502600
200,0.439500
300,0.383900
400,0.359000
500,0.306600
600,0.340400
700,0.344400
800,0.317500
900,0.309700
1000,0.231400


<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=9970, training_loss=0.08740493117597183, metrics={'train_runtime': 2101.0629, 'train_samples_per_second': 37.943, 'train_steps_per_second': 4.745, 'total_flos': 5243803333324800.0, 'train_loss': 0.08740493117597183, 'epoch': 10.0})

In [26]:
# 모델 평가
trainer.evaluate(eval_dataset=test_dataset)

<ipython-input-18-32887a5ea9b7>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.7632253766059875,
 'eval_accuracy': 0.9083625438157236,
 'eval_f1': 0.9080863887493722,
 'eval_precision': 0.8986083499005965,
 'eval_recall': 0.917766497461929,
 'eval_runtime': 13.3182,
 'eval_samples_per_second': 149.945,
 'eval_steps_per_second': 2.403,
 'epoch': 10.0}

In [27]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
  # 평가모드로 변경
  model.eval()

  # 입력된 문장 토크나이징
  tokenized_sent = tokenizer(
      sent,
      return_tensors="pt",
      truncation=True,
      add_special_tokens=True,
      max_length=128
  )

  # 모델이 위치한 GPU로 이동
  tokenized_sent.to(device)


  # 예측
  with torch.no_grad():
    outputs = model(
        input_ids=tokenized_sent["input_ids"],
        attention_mask=tokenized_sent["attention_mask"],
        token_type_ids=tokenized_sent["token_type_ids"]
    )

  # 결과 return
  logits = outputs[0]
  logits = logits.detach().cpu()
  result = logits.argmax(-1)
  if result == 0:
    result = " >> 악성댓글 👿"
  elif result == 1:
    result = " >> 정상댓글 😀"
  return result

In [ ]:
# 0 입력시 종료
while True:
  sentence = input("댓글을 입력해주세요: ")
  if sentence == "0" :
    break

  print(sentence_predict(sentence))
  print("\n")

댓글을 입력해주세요: 간나새끼
 >> 악성댓글 👿


댓글을 입력해주세요: 나쁜새끼
 >> 악성댓글 👿


댓글을 입력해주세요: 너는 천사야
 >> 정상댓글 😀


댓글을 입력해주세요: 반가워
 >> 정상댓글 😀


